In [6]:
!pip -q install scapy pandas

import pandas as pd
from scapy.all import rdpcap
from scapy.layers.inet import IP, UDP
from scapy.layers.dhcp import DHCP, BOOTP

PCAP_PATH = "/content/dhcp.pcapng"  # путь к файлу

packets = rdpcap(PCAP_PATH)

rows = []

for pkt in packets:
    if pkt.haslayer(DHCP) and pkt.haslayer(IP):
        dhcp = pkt[DHCP]
        ip = pkt[IP]
        bootp = pkt[BOOTP]

        # тип DHCP сообщения
        dhcp_type = None
        for option in dhcp.options:
            if isinstance(option, tuple) and option[0] == "message-type":
                dhcp_type = option[1]

        rows.append(
            {
                "time": pd.to_datetime(float(pkt.time), unit="s"),
                "src_ip": ip.src,
                "dst_ip": ip.dst,
                "transaction_id": hex(bootp.xid),
                "client_mac": bootp.chaddr[:6].hex(":"),
                "your_ip": bootp.yiaddr,
                "dhcp_type": dhcp_type,
            }
        )

df = pd.DataFrame(rows)
df

,time,src_ip,dst_ip,transaction_id,client_mac,your_ip,dhcp_type
0,2004-12-05 19:16:24.317452908,0.0.0.0,255.255.255.255,0x3d1d,00:0b:82:01:fc:42,0.0.0.0,1
1,2004-12-05 19:16:24.317748070,192.168.0.1,192.168.0.10,0x3d1d,00:0b:82:01:fc:42,192.168.0.10,2
2,2004-12-05 19:16:24.387484074,0.0.0.0,255.255.255.255,0x3d1e,00:0b:82:01:fc:42,0.0.0.0,3
3,2004-12-05 19:16:24.387798071,192.168.0.1,192.168.0.10,0x3d1e,00:0b:82:01:fc:42,192.168.0.10,5
